# 3.How to find the CSL/DSC lattices of a Σ-rotation using GBpy!
-----------------------------------------------------------------------------------------

In [1]:
import byxtal
import numpy as np
import os
import inspect
import pickle

ModuleNotFoundError: No module named 'byxtal'

## 3.1. Finding the Σ-rotations

While Σ-rotations are widely known and used in crystallography they are not trivial to build. Therefore we have computed a wide range of Σ-rotations and have included them inside the byxtal package in a separate folder named pkl_files. If you have installed the byxtal package using the pip install byxtal command or if you have cloned the GBpy github repository on Github you have received the pkl_files folder. To access the pkl_files folder, you need to know where the byxtal package resides on your system. The following steps will help you navigate through the byxtal package folder structure. If you already have the rotation matrix (3x3 matrix) associated with the Σ-rotation igonre the following steps.

In [2]:
gbpy_dir = os.path.dirname((inspect.getfile(GBpy)))
print(gbpy_dir)

NameError: name 'os' is not defined

The variable gbpy_dir is a string contaning the path to where GBpy resides on your system. pkl_files is a subfolder of the gbpy_dir. You can get the list of the files that are included in the pkl_files folder using the following command:

In [3]:
pkl_dir = gbpy_dir + '/pkl_files'

NameError: name 'gbpy_dir' is not defined

os.listdir(pkl_dir)

The common Σ-rotations of cubic lattices are saved in a file named "cF_Id_csl_common_rotations.pkl". You can find Σ-rotations of other lattices in the same folder similarly (i.e. hexagnal(hP), tetragonal(tP), etc.), however in this tutorial the focus is on face-centered cubic (f.c.c) lattice. To access the contents of "cF_Id_csl_common_rotations.pkl" use the following commands:

In [5]:
pkl_file = pkl_dir + '/cF_Id_csl_common_rotations.pkl'
pkl_content = pickle.load(open(pkl_file))

NameError: name 'pkl_dir' is not defined

The variable "pkl_content" contains first 200 common Σ-rotations of cubic lattices, stored as a dictionary data structure. Each entry in the dictionary can be access by specifying the __key__ to that entry. The key for each sigma rotation is a __string__ with the Σ-number. We would like to emphasize that the key to each entry has to be a string. For instance for Σ

3 you need to pass '3' to the pkl_content as the key.

It is very important to note that all the rotation matrices are expressed in __primitive bases__ of the associated lattices. To use the rotation matrices in an orthogonal basis one needs to perform similarity transformation on the provided rotation matrices. We will demonstrate an example of such transformation later in this tutorial. Luckily for cubic lattices the Σ

-rotations remain constant in both primitive and orthogonal bases.

To avoid the truncation error we have stored the Numerator (saved as 'N') and the Denominator (saved as 'D') of each Σ
-rotation separately. To construct the rotation matrix associated with each Σ-rotation you need to divide the elements of __N__ by elements of __D__. The result will be a 3x3 rotation matrix which will be a numpy array. The following steps shows how you can construct the rotation matrix associated with the Σ3 rotation:

In [7]:
pkl_content['3']

NameError: name 'pkl_content' is not defined

In [8]:
N3 = pkl_content['3']['N'][0]
D3 = pkl_content['3']['D'][0]
Sigma3 = N3/D3
print(Sigma3)

NameError: name 'pkl_content' is not defined

Note that some of the Sigmas have more than one Sigma rotation. For instance Σ17 has two Σ-rotations often referred to as Σ17-a and Σ17-b. In such cases you will find two N matrices and two D matrices associated with each rotation. Use the following to access each N and D matrix.

In [9]:
pkl_content['17']

NameError: name 'pkl_content' is not defined

In [10]:
N17_a = pkl_content['17']['N'][0]
D17_a = pkl_content['17']['D'][0]
Sigma17_a = N17_a/D17_a
print 'Sigma17_a =', Sigma17_a, '\n'

N17_b = pkl_content['17']['N'][1]
D17_b = pkl_content['17']['D'][1]
Sigma17_b = N17_b/D17_b
print 'Sigma17_b =', Sigma17_b,

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('Sigma17_a =', Sigma17_a, '\n')? (<ipython-input-10-18cbe0736c7d>, line 4)

## 3.2. Changing the Basis of a Transformation Matrix

So far we have been able to obtain the rotation matrix associated with the Σ3 rotation which is stored in the variable __Sigma3__. As mentioned in __section 1.1__, it is very important to recognize that the obtained rotation matrices are expressed in primitive basis of the lattice. To use the rotation matrices in an orthogonal basis one needs to perform similarity transformation on the provided rotation matrices.

By convention we use the __p__ for primitive basis and __po__ for orthogonal basis of a lattice.

If we define the transformation matrix of the Σ
3 (that transforms crystal p1 to p2) in the primitive basis as Tpp1→p2, same transformation in orthogonal basis of the lattice (e.g. supercell lattice) will be defined as Tpop1→p2

and can be found using the similarity transformation as:

$\mathcal{T^{po}_{p1→p2}=Λ^{po}_{p} × T^{p}_{p1→p2} × Λ^{p}_{po}}$

where $\mathcal{Λ^{po}_{p}}$ is a 3x3 matrix which defines the components of the basis of the primitve lattice in an orthogonal reference frame attached to the same lattice. Therefore, Λpop is a constant for each lattice. Λpop for various lattices can be obtained from the lattice class in byxtal. For example, the components of a primitive f.c.c. lattice with lattice parameter a=1.0Å, $\mathcal{Λ^{po}_{p}}$, is defined as:

In [ ]:
L_p_po = 1.0*np.array([[0.,0.5,0.5],[0.5,0.,0.5],[0.5,0.5,0.]])
L_p_po

and $\mathcal{Λ^{po}_{p}}$ is the inverse of $\mathcal{Λ^{p}_{po}}$:

In [ ]:
L_po_p = np.linalg.inv(L_p_po)
L_po_p

Therefore using similarity transformation, $\mathcal{T^{po}_{p1→p2}}$ can be obtained as:

In [ ]:
T_p1top2_po = np.dot(L_p_po, np.dot(Sigma3, L_po_p)).reshape(1,3,3)[0]
T_p1top2_po

As mentioned in section 1.1. it can be observed that $\mathcal{T^{po}_{p1→p2}}$ and $\mathcal{T^{p}_{p1→p2}}$ are the same for cubic lattices.

## 3.3. Finding the CSL and DSC Lattices of a Σ3 rotation:

To compute the CSL and DSC lattices, __byxtal__ only requires the  $\mathcal{T^{p}_{p1→p2}}$ that can be directly accessed from the provided pkl files. It automatically performs the similarity transformation using the $\mathcal{Λ^{po}_{p}}$ that can be obtained from the lattice class.

We have stored the $\mathcal{Λ^{po}_{p}}$ in __L_p_po__ and $\mathcal{T^{p}_{p1→p2}}$ in __Sigma3__ that are shown in the following:

In [11]:
L_p_po

NameError: name 'L_p_po' is not defined

In [ ]:
Sigma3

To find the CSL and DSC lattices the user should use the function __find_csl_dsc__. This function is available in a file with the same name. To use this function first you must import byxtal as a package or directly import the function from the file.

__find_csl_dsc__ needs two inputs and provides the CSL and DSC lattices in primitive basis. The syntax is as follows:

$\mathcal{Λ^{p1}_{CSL}, Λ^{p1}_{DSC}}$ = __find_csl_dsc__ ($\mathcal{Λ^{po}_p, T^p_{p1→p2}})$


Follow the steps below to obtain the CSL and DSC lattices using __byxtal__:

First import the appropriate module from __byxtal__. Next call the __find_csl_dsc__ function with the following syntax:

In [ ]:
import GBpy.find_csl_dsc as gb_csl
L_csl_p1, L_dsc_p1 = gb_csl.find_csl_dsc(L_p_po, Sigma3)

where $\mathcal{Λ^{p1}_{CSL}}$ is obtained as:

In [ ]:
print(L_csl_p1)

and $\mathcal{Λ^{p1}_{DSC}}$ is obtained as:

In [ ]:
print(L_dsc_p1)

To find the $\mathcal{Λ^{po1}_{CSL}}$ and $\mathcal{Λ^{po1}_{DSC}}$ , one must change the reference frame to the orthogonal basis as follows:

$\mathcal{Λ^{po1}_{CSL}}$= $\mathcal{Λ^{po}_{p}}$ × $\mathcal{Λ^{po1}_{DSC}}$

Hence we can obtain the $\mathcal{Λ^{po1}_{CSL}}$ as:

In [ ]:
L_csl_po1 = np.dot(L_p_po, L_csl_p1)
print(L_csl_po1)

Also we have:

$\mathcal{Λ^{po1}_{DSC}}$ = $\mathcal{Λ^{po}_{p}}$ × $\mathcal{Λ^{p1}_{DSC}}$

Therefore we can obtain the $\mathcal{Λ^{po1}_{DSC}}$ as:

In [ ]:
L_dsc_po1 = np.dot(L_p_po, L_dsc_p1)
print L_dsc_po1

The $\mathcal{Λ^{po1}_{CSL}}$ provides a basis for the __coincidence sites__ of a bicrystal that have a misorientation of the specified Σ (e.g. Σ3 in this case). Note that the basis obtained $\mathcal{Λ^{po1}_{CSL}}$ is in reduced form (using lll-reduction algorithm) and therefore the length of the basis vectors are as close as possible. Each column of the Λpo1CSL

contains one of the basis vectors of the CSL lattice. These basis vectors of CSL the lattice are of paramount importance in simulation of interfaces and are used to construct a periodic simulation cell.

The $\mathcal{Λ^{po1}_{DSC}}$ provides a basis for the __displacement shift lattice__ of a bicrystal that have a misorientation of the specified Σ (e.g. Σ3 in this case). The basis vectors of a DSC lattice provide unique displacement intervals in space in which the coincidence pattern of the bicrystal remains preserved. In other words any displacement in one of the bicrystals along the DSC basis vectors and with a magnitude of integer multiples of the norm of the DSC basis vectors, preserves the CSL pattern. This property of the DSC lattice is of great importance in energy minimization of interfaces.